In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC



# RandomForestClassifier, DecisionTreeClassifier, KNeighborsClassifier, LinearDiscriminantAnalysis, GaussianNB,SVC

In [2]:
protein_df = pd.read_csv('../data/raw/ProteinAndPathologyQuantifications.csv')
Donor_df = pd.read_csv('../data/raw/DonorInformation.csv')
Stains_df = pd.read_csv('../data/raw/DescriptionOfStains.csv')
columns_samples_df = pd.read_csv('../data/raw/gene_expression_matrix_2016-03-03/columns-samples.csv')
rows_genes_df = pd.read_csv('../data/raw/gene_expression_matrix_2016-03-03/rows-genes.csv')


In [3]:
columns_samples_df

,rnaseq_profile_id,donor_id,donor_name,specimen_id,specimen_name,rna_well_id,polygon_id,structure_id,structure_acronym,structure_color,structure_name,hemisphere
0,488395315,309335467,H14.09.030,309357843,H14.09.030.TCx.01,395325172,320817998,10235,TCx,#ebbfd0,temporal neocortex,left
1,496100277,309335441,H14.09.004,309357624,H14.09.004.PCx.01,320630866,310967169,10557,FWM,#f2f1f0,white matter of forebrain,right
2,496100278,309335438,H14.09.001,309357596,H14.09.001.PCx.01,320630834,310790571,10557,FWM,#f2f1f0,white matter of forebrain,left
3,496100279,309335438,H14.09.001,309357599,H14.09.001.TCx.01,320630838,310790522,10235,TCx,#ebbfd0,temporal neocortex,left
4,496100281,309335439,H14.09.002,309357603,H14.09.002.HIP.01,320630842,310790372,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right
...,...,...,...,...,...,...,...,...,...,...,...,...
372,496100667,467056391,H15.09.103,467179071,H15.09.103.TCx.01,482655826,480366830,10235,TCx,#ebbfd0,temporal neocortex,right
373,496100669,467056391,H15.09.103,467179068,H15.09.103.PCx.01,482655822,480363830,10557,FWM,#f2f1f0,white matter of forebrain,right
374,496100670,467056406,H15.09.107,467179104,H15.09.107.TCx.01,482655780,480363840,10235,TCx,#ebbfd0,temporal neocortex,right
375,496100671,467056391,H15.09.103,467179065,H15.09.103.HIP.01,482655820,480366825,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right


In [4]:
rows_genes_df

,gene_id,chromosome,gene_entrez_id,gene_symbol,gene_name
0,499304660,1,100287102,DDX11L1,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
1,499304661,1,653635,WASH7P,WAS protein family homolog 7 pseudogene
2,499304662,1,102466751,MIR6859-1,microRNA 6859-1
3,499304663,1,100302278,MIR1302-2,microRNA 1302-2
4,499304664,1,645520,FAM138A,"family with sequence similarity 138, member A"
...,...,...,...,...,...
50276,499355059,MT,4541,ND6,NADH dehydrogenase subunit 6
50277,499355060,MT,4556,TRNE,tRNA-Glu
50278,499355061,MT,4519,CYTB,cytochrome b
50279,499355062,MT,4576,TRNT,tRNA-Thr


In [5]:
len(rows_genes_df.gene_symbol.unique())

50281

In [6]:
gene_symbol_lst = list(rows_genes_df.gene_symbol)

In [7]:
pd.set_option('display.max_colwidth', None)

Stains_df

,api_class_name,internal_name,description,treatment_id,full_name,gene_id,method,display_name,long_display_name
0,ApiTbiDonorMetric,ihc_a_beta,Percentage of area covered by Aβ immunreactivity on fresh frozen tissue (25μm); identifies both Aβ40 and Aβ42.,300239718.0,Beta-amyloid immunohistochemistry (fresh frozen tissue): areal density,NaN,IHC: Fresh Frozen,Aβ(fresh),Aβ IHC density (fresh)
1,TbiNonquantifiedImageStains,h_e_lfb_ffpe,Hematoxylin identifies cell nuclei; Eosin stains cytoplasm and some cytoplasmic inclusions; Luxol Fast Blue stains myelin in tissue (5 μm).,471133394.0,Hematoxylin and eosin-Luxol Fast Blue stains,NaN,"Formalin Fixed, Paraffin Embedded",H&E-LFB(FFPE),H&E-LFB stain (FFPE)
2,ApiTbiDonorMetric,ihc_a_syn,Percentage of area covered by α-synuclein immunreactivity on fresh frozen tissue (25μm); identifies Lewy bodies and Lewy neurites.,300239725.0,Alpha-synuclein immunohistochemistry (fresh frozen tissue): areal density,NaN,IHC: Fresh Frozen,α-syn(fresh),α-syn IHC density (fresh)
3,ApiTbiDonorDetail,root_specimen_external_name,UW autopsy number,NaN,Top Specimen External Name,NaN,NaN,NaN,NaN
4,ApiTbiDonorMetric,ifn_g_pg_per_mg,"A prototype pro-inflammatory cytokine, critical for innate and adaptive immunity against viral and some bacterial infection via activation of macrophages and induction of class II MHC expression.",NaN,Interferon-gamma,NaN,Luminex,IFNγ,IFNγ protein quant.
...,...,...,...,...,...,...,...,...,...
57,TbiNonquantifiedImageStains,ish_gfap,"ISH of GFAP, targeting activated astrocytes.",1.0,Glial fibrillary acidic protein (ISH),2654.0,ISH,GFAP,GFAP
58,ApiTbiDonorDetail,braak,Staging of neurofibrillary tangles (NFTs) from 0 (none) to 6 (extensive neocortical NFTs) based on Braak method of staging.,NaN,Braak stage,NaN,NaN,Braak stage,Braak stage
59,ApiTbiDonorMetric,tnf_a_pg_per_mg,"A pro-inflammatory cytokine, produced primarily by macrophages, regulates inflammatory response, especially in acute phase.",NaN,Tumor necrosis factor alpha,NaN,Luminex,TNFα,TNFα protein quant.
60,ApiTbiDonorDetail,hispanic,Hispanic or non-hispanic,NaN,Hispanic,NaN,NaN,Ethnicity,Ethnicity


In [8]:
Stains_df.gene_id.unique()

array([   nan,   358., 36308.,  6493.,  4310.,  5617.,  2654.])

In [9]:
Stains_df['internal_name'].unique()

array(['ihc_a_beta', 'h_e_lfb_ffpe', 'ihc_a_syn',
       'root_specimen_external_name', 'ifn_g_pg_per_mg', 'race', 'name',
       'ish_aqp4', 'ish_slc17a7', 'ever_tbi_w_loc', 'rantes_pg_per_mg',
       'ihc_tau2_ffpe', 'external_donor_name', 'isoprostane_pg_per_mg',
       'act_demented', 'bdnf_pg_per_mg', 'ptau_over_tau_ratio',
       'ish_slc6a1', 'ptau_ng_per_mg', 'ish_mobp', 'mip_1a_pg_per_mg',
       'il_4_pg_per_mg', 'pmi_hrs', 'ab42_over_ab40_ratio',
       'ihc_a_beta_ffpe', 'ab42_pg_per_mg', 'mcp_1_pg_per_mg',
       'nincds_arda_diagnosis', 'ihc_at8', 'education_years',
       'apo_e4_allele', 'nissl', 'longest_loc_duration', 'donor_id',
       'ab40_pg_per_mg', 'ihc_iba1_ffpe', 'ihc_a_syn_ffpe', 'ish_reln',
       'num_tbi_w_loc', 'dsm_iv_clinical_diagnosis',
       'ab40_over_ab42_ratio', 'control_set', 'il_1b_pg_per_mg',
       'ihc_ptdp_43_ffpe', 'sex', 'cerad', 'il_7_pg_per_mg',
       'il_6_pg_per_mg', 'ihc_at8_ffpe', 'tau_ng_per_mg', 'nia_reagan',
       'a_syn_pg_per_

In [10]:
protein_df

,donor_id,donor_name,structure_id,structure_acronym,ihc_a_syn,ihc_tau2_ffpe,ihc_at8_ffpe,ihc_at8,ihc_ptdp_43_ffpe,ihc_a_beta_ffpe,...,il_4_pg_per_mg,rantes_pg_per_mg,ab40_pg_per_mg,a_syn_pg_per_mg,ifn_g_pg_per_mg,mcp_1_pg_per_mg,bdnf_pg_per_mg,mip_1a_pg_per_mg,il_7_pg_per_mg,ab42_pg_per_mg
0,309335467,H14.09.030,10557,FWM,0.000078,0.002358,0.001137,0.000110,0.001259,0.008335,...,2.68,15.99,0.736100,0.122288,1.47,20.78,5.03736,9.38,11.78,523.292251
1,309335480,H14.09.043,10208,PCx,0.000063,0.002762,0.001272,0.000164,0.002354,0.005047,...,3.02,24.04,0.669094,0.111962,1.44,40.40,4.95462,8.10,45.02,81.493875
2,309335493,H14.09.056,10557,FWM,0.000064,0.003468,0.013787,0.016023,0.001708,0.007365,...,0.80,129.80,0.736100,0.091084,0.54,46.88,5.88409,27.00,15.82,470.734514
3,326765668,H14.09.081,10557,FWM,0.000049,0.003035,0.001707,0.000137,0.001729,0.004046,...,1.30,15.16,8.995575,0.054076,0.76,10.34,16.13524,8.06,24.22,568.368571
4,326765668,H14.09.081,10235,TCx,0.000080,0.002088,0.004489,0.000062,0.001513,0.015809,...,0.00,17.90,95.931000,0.030000,1.22,21.38,0.03000,10.52,23.72,438.863263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,326765653,H14.09.066,10208,PCx,0.000046,0.001989,0.000533,0.000082,0.001130,0.003128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,309335452,H14.09.015,10294,HIP,0.000089,0.001918,0.004685,0.000923,0.001181,0.003483,...,0.00,5.94,0.574500,0.248816,0.00,7.16,2.94819,17.82,1.38,0.054413
374,309335489,H14.09.052,10294,HIP,0.000066,0.002275,0.006337,0.009560,0.001395,0.001710,...,0.00,7.62,0.900050,0.183510,0.00,3.14,10.56419,21.96,0.60,0.181375
375,309335458,H14.09.021,10208,PCx,0.000793,0.004384,0.001266,0.000093,0.002505,0.009795,...,0.02,4.88,0.444250,0.045202,0.00,7.20,2.83064,0.00,12.46,205.886650


In [11]:
protein_df.structure_acronym.unique()

array(['FWM', 'PCx', 'TCx', 'HIP'], dtype=object)

In [12]:
proteins_list = protein_df.columns

In [13]:
proteins_list

Index(['donor_id', 'donor_name', 'structure_id', 'structure_acronym',
       'ihc_a_syn', 'ihc_tau2_ffpe', 'ihc_at8_ffpe', 'ihc_at8',
       'ihc_ptdp_43_ffpe', 'ihc_a_beta_ffpe', 'ihc_a_beta', 'ihc_iba1_ffpe',
       'ihc_gfap_ffpe', 'ptau_ng_per_mg', 'vegf_pg_per_mg',
       'ab42_over_ab40_ratio', 'tnf_a_pg_per_mg', 'tau_ng_per_mg',
       'il_10_pg_per_mg', 'isoprostane_pg_per_mg', 'il_6_pg_per_mg',
       'il_1b_pg_per_mg', 'ptau_over_tau_ratio', 'il_4_pg_per_mg',
       'rantes_pg_per_mg', 'ab40_pg_per_mg', 'a_syn_pg_per_mg',
       'ifn_g_pg_per_mg', 'mcp_1_pg_per_mg', 'bdnf_pg_per_mg',
       'mip_1a_pg_per_mg', 'il_7_pg_per_mg', 'ab42_pg_per_mg'],
      dtype='object')

In [14]:
proteins_list = proteins_list[4:]

In [15]:
proteins_list

Index(['ihc_a_syn', 'ihc_tau2_ffpe', 'ihc_at8_ffpe', 'ihc_at8',
       'ihc_ptdp_43_ffpe', 'ihc_a_beta_ffpe', 'ihc_a_beta', 'ihc_iba1_ffpe',
       'ihc_gfap_ffpe', 'ptau_ng_per_mg', 'vegf_pg_per_mg',
       'ab42_over_ab40_ratio', 'tnf_a_pg_per_mg', 'tau_ng_per_mg',
       'il_10_pg_per_mg', 'isoprostane_pg_per_mg', 'il_6_pg_per_mg',
       'il_1b_pg_per_mg', 'ptau_over_tau_ratio', 'il_4_pg_per_mg',
       'rantes_pg_per_mg', 'ab40_pg_per_mg', 'a_syn_pg_per_mg',
       'ifn_g_pg_per_mg', 'mcp_1_pg_per_mg', 'bdnf_pg_per_mg',
       'mip_1a_pg_per_mg', 'il_7_pg_per_mg', 'ab42_pg_per_mg'],
      dtype='object')

In [16]:
Stains_name_description_df = Stains_df[['internal_name','description']]

In [17]:
Stains_name_description_df

,internal_name,description
0,ihc_a_beta,Percentage of area covered by Aβ immunreactivity on fresh frozen tissue (25μm); identifies both Aβ40 and Aβ42.
1,h_e_lfb_ffpe,Hematoxylin identifies cell nuclei; Eosin stains cytoplasm and some cytoplasmic inclusions; Luxol Fast Blue stains myelin in tissue (5 μm).
2,ihc_a_syn,Percentage of area covered by α-synuclein immunreactivity on fresh frozen tissue (25μm); identifies Lewy bodies and Lewy neurites.
3,root_specimen_external_name,UW autopsy number
4,ifn_g_pg_per_mg,"A prototype pro-inflammatory cytokine, critical for innate and adaptive immunity against viral and some bacterial infection via activation of macrophages and induction of class II MHC expression."
...,...,...
57,ish_gfap,"ISH of GFAP, targeting activated astrocytes."
58,braak,Staging of neurofibrillary tangles (NFTs) from 0 (none) to 6 (extensive neocortical NFTs) based on Braak method of staging.
59,tnf_a_pg_per_mg,"A pro-inflammatory cytokine, produced primarily by macrophages, regulates inflammatory response, especially in acute phase."
60,hispanic,Hispanic or non-hispanic


In [18]:
protein_descriptions_df = Stains_name_description_df[Stains_name_description_df['internal_name'].isin(proteins_list)]

In [19]:
protein_descriptions_df

,internal_name,description
0,ihc_a_beta,Percentage of area covered by Aβ immunreactivity on fresh frozen tissue (25μm); identifies both Aβ40 and Aβ42.
2,ihc_a_syn,Percentage of area covered by α-synuclein immunreactivity on fresh frozen tissue (25μm); identifies Lewy bodies and Lewy neurites.
4,ifn_g_pg_per_mg,"A prototype pro-inflammatory cytokine, critical for innate and adaptive immunity against viral and some bacterial infection via activation of macrophages and induction of class II MHC expression."
10,rantes_pg_per_mg,A chemokine belonging to the CC chemokine subfamily produced by epithelial and mesodermal cells. It plays an active role in recruiting leukocytes into inflammatory sites.
11,ihc_tau2_ffpe,"Percentage of area covered by Tau2 immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies mature neurofibrillary tangles and dystrophic neurites."
13,isoprostane_pg_per_mg,"Prostaglandin-like compounds produced by the reaction of free-radicals with arachidonic acid, they are considered reliable biomarkers and mediators of oxidative stress."
15,bdnf_pg_per_mg,"A neurotrophic factor primarily produced in the brain, it promotes the growth and differentiation of neurons and synapses."
16,ptau_over_tau_ratio,Ratio of tau phosphorylated at threonine-181 to full length tau.
18,ptau_ng_per_mg,Tau abnormally phosphorylated at threonine 181; abnormally phosphorylated tau plays an important role in tau aggregation and the pathological process in Alzheimer's disease; neurofibrillary tangles are comprised of hyperphosphorylated tau.
20,mip_1a_pg_per_mg,A chemokine of the CC subfamily with chemotactic and pro-inflammatory effects; crucial for immune responses.


In [20]:
len(protein_descriptions_df)

29

# Proteins


In [21]:
proteins_dict = {key: None for key in proteins_list}


In [22]:
proteins_dict['ihc_a_beta'] = {'Protein':'Amyloid β (ihc)', 
                               'Description': 'Percentage of area covered by Aβ immunreactivity on fresh frozen tissue (25μm); identifies both Aβ40 and Aβ42',
                               'Articles': ['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1824787/'],
                               'Gene': 'APP'}
proteins_dict['ihc_a_syn'] = {'Protein':'α-synuclein (ihc)',
                              'Description':'Percentage of area covered by α-synuclein immunreactivity on fresh frozen tissue (25μm); identifies Lewy bodies and Lewy neurites.',
                              'Articles': None,
                              'Gene':'SNCA'}
proteins_dict['ifn_g_pg_per_mg'] = {'Protein': 'Interferon gamma (conc)',
                                    'Description': 'Prototype pro-inflammatory cytokine, critical for innate and adaptive immunity',
                                    'Articles': None,
                                    'Gene': 'IFN'}
proteins_dict['rantes_pg_per_mg'] = {'Protein':'RANTES (conc)',
                                     'Description': 'Recruit leukocytes into inflammatory sites',
                                     'Articles': None,
                                     'Gene': 'CCL5'}
proteins_dict['ihc_tau2_ffpe'] = {'Protein': 'Tau2 (ihc-ffpe)',
                                  'Description': 'Percentage of area covered by Tau2 immunreactivity',
                                  'Articles': None,
                                 'Gene': 'MAPT'}

proteins_dict['isoprostane_pg_per_mg'] = {'Protein': 'Isoprostane (conc)',
                                         'Description':'Prostaglandin-like compounds considered reliable biomarkers of oxidative stress',
                                         'Articles': None,
                                         'Gene': None}

proteins_dict['bdnf_pg_per_mg'] = {'Protein': 'Brain-derived neurotrophic factor (conc)',
                                         'Description':'Primarily produced in the brain, promotes growth and differentiation of neurons and synapses.',
                                         'Articles': ['https://translationalneurodegeneration.biomedcentral.com/articles/10.1186/s40035-022-00279-0'],
                                         'Gene': 'BDNF'}

proteins_dict['ptau_over_tau_ratio'] = {'Protein': 'Phosphorylated Tau (ratio)',
                                         'Description':' Ratio of tau phosphorylated at threonine-181 to full length tau.',
                                         'Articles': None,
                                         'Gene': 'MAPT'}

proteins_dict['ptau_ng_per_mg'] = {'Protein': 'Phosphorylated Tau (conc)',
                                    'Description':'Tau abnormally phosphorylated at threonine 181; abnormally phosphorylated tau plays an important role in tau aggregation and the pathological process in Alzheimers disease; neurofibrillary tangles are comprised of hyperphosphorylated tau.',
                                    'Articles': None,
                                    'Gene': 'MAPT'}

proteins_dict['mip_1a_pg_per_mg'] = {'Protein': 'Macrophage Inflammatory Protein-1 Alpha (conc)',
                                    'Description':' A chemokine of the CC subfamily with chemotactic and pro-inflammatory effects; crucial for immune responses.',
                                    'Articles': ['https://pubmed.ncbi.nlm.nih.gov/17656823/'],
                                    'Gene': 'CCL3'}

proteins_dict['il_4_pg_per_mg'] = {'Protein': 'Interleukin 4 (conc)',
                    'Description':'A pleiotropic cytokine, produced by activated T cells and mast cells, stimulates B-cell activation and T-cell proliferation. It plays a dominant role in development of allergic inflammation.',
                    'Articles': None,
                    'Gene': 'IL4'}

proteins_dict['ab42_over_ab40_ratio'] = {'Protein': 'Amyloid β (ratio)',
                    'Description':'Ratio of Aβ40 to Aβ42',
                    'Articles': None,
                    'Gene': 'APP'}

proteins_dict['ihc_a_beta_ffpe'] = {'Protein': 'Amyloid β (ihc-ffpe)',
                    'Description':'Percentage of area covered by Aβ immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies both Aβ40 and Aβ42.',
                    'Articles': None,
                    'Gene': 'APP'}

proteins_dict['ab42_pg_per_mg'] = {'Protein': 'Amyloid β 42 (conc)',
                    'Description':'Percentage of area covered by Aβ immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies both Aβ40 and Aβ42.',
                    'Articles': None,
                    'Gene': 'APP'}

proteins_dict['mcp_1_pg_per_mg'] = {'Protein': 'Monocyte Chemoattractant Protein-1 (conc)',
                    'Description':'A chemokine of the CC subfamily that displays chemotactic activity for monocytes primarily to the sites of inflammation.',
                    'Articles': None,
                    'Gene': 'CCL2'}

proteins_dict['ihc_at8'] = {'Protein': 'AT8 (ihc)',
                    'Description':'Anti-Tau Antibody. Percentage of area covered by AT8 immunreactivity on fresh frozen tissue (25μm); identifies immature and mature neurofibrillary tangles, formed of tau phosphorylated at S202 and T205. Also identifies neurites, glial tangles, and inclusion bodies comprised of phosphorylated tau protein.',
                    'Articles': None,
                    'Gene': 'MAPT'}

proteins_dict['ab40_pg_per_mg'] = {'Protein': 'Amyloid β 40 (conc)',
                    'Description':'40-amino acid beta-amyloid peptide derived from the amyloid precursor protein (APP); found in amyloid plaques in aging and Alzheimers disease.',
                    'Articles': None,
                    'Gene': 'APP'}

proteins_dict['ihc_iba1_ffpe'] = {'Protein': 'Ionized calcium binding adaptor molecule 1 (ihc-ffpe)',
                    'Description':'Percentage of area covered by IBA1 immunreactivity on formalin fixed, paraffin embedded tissue (5μm).',
                    'Articles': None,
                    'Gene': 'AIF1'}

proteins_dict['il_1b_pg_per_mg'] = {'Protein': 'Interleukin 1b (conc)',
                    'Description':'A pro-inflammatory cytokine protein produced in response to inflammatory stimuli, it is an important mediator of the inflammatory response. Involved in cell proliferation, differentiation and apoptosis.',
                    'Articles': None,
                    'Gene': 'IL1B'}

proteins_dict['ihc_ptdp_43_ffpe'] = {'Protein': 'phospho-TDP43 (TAR DNA binding protein) (ihc-ffpe)',
                    'Description':'Percentage of area covered by phospho-TDP43 (TAR DNA binding protein) immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies abnormally accumulated TDP43.',
                    'Articles': None,
                    'Gene': 'TARDBP'}

proteins_dict['il_7_pg_per_mg'] = {'Protein': 'Interleukin 7 (conc)',
                    'Description':'A hematopoietic growth factor that stimulates the differentiation of pluripotent hematopoietic stem cells into lymphoid progenitor cells. It is important for lymphocyte differentiation, proliferation, and survival.',
                    'Articles': None,
                    'Gene': 'IL7'}

proteins_dict['il_6_pg_per_mg'] = {'Protein': 'Interleukin 6 (conc)',
                    'Description':'A pro-inflammatory cytokine that plays an important role in acute response of inflammation and B-cell maturation.',
                    'Articles': None,
                    'Gene': 'IL6'}

proteins_dict['ihc_at8_ffpe'] = {'Protein': 'AT8 (ihc-ffpe)',
                    'Description':'Percentage of area covered by AT8 immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies immature and mature neurofibrillary tangles, formed of tau phosphorylated at S202 and T205. Also identifies neurites, glial tangles, and inclusion bodies comprised of phosphorylated tau protein.',
                    'Articles': None,
                    'Gene': 'MAPT'}

proteins_dict['tau_ng_per_mg'] = {'Protein': 'Tau (conc)',
                    'Description':'Microtubule binding protein predominantly expressed in neurons in the brain.',
                    'Articles': None,
                    'Gene': 'MAPT'}

proteins_dict['a_syn_pg_per_mg'] = {'Protein': 'α-synuclein (conc)',
                    'Description':'An abundant protein in presynaptic terminals in the brain, it is a major component of Lewy bodies, the pathological hallmark of Parkinson's disease (PD). Mutant forms of α-SNCA are associated with familial PD. It may also play a role in development of sporadic PD.',
                    'Articles': None,
                    'Gene': 'SNCA'}

proteins_dict['ihc_gfap_ffpe'] = {'Protein': 'glial fibrillary acidic protein (ihc-ffpe)',
                    'Description':'Percentage of area covered by GFAP immunreactivity on formalin fixed, paraffin embedded tissue (5μm); identifies activated astrocytes.',
                    'Articles': None,
                    'Gene': 'GFAP'}
proteins_dict['vegf_pg_per_mg'] = {'Protein': 'VEGF (conc)',
                    'Description':'A growth factor that stimulates vasculogenesis and angiogenesis.',
                    'Articles': None,
                    'Gene': 'VEGF'}

proteins_dict['il_10_pg_per_mg'] = {'Protein': 'Interleukin 10 (conc)',
                    'Description':'An anti-inflammatory cytokine produced primarily via downregulation of the expression of Th1 cytokines, MHC class II antigens, and the activation of macrophages.',
                    'Articles': None,
                    'Gene': 'IL10'}

proteins_dict['tnf_a_pg_per_mg'] = {'Protein': '',
                    'Description':'A pro-inflammatory cytokine, produced primarily by macrophages, regulates inflammatory response, especially in acute phase.',
                    'Articles': None,
                    'Gene': 'TNF'}

In [41]:
protein_genes_lst = ['APP','SNCA','IFN','CCL5','MAPT','TAU','BDNF','CCL3','IL4','CCL2','AIF1','IL1B','TARDBP',
                    'IL7','IL6','GFAP','VEGF', 'IL10', 'TNF']


In [42]:
for gene in protein_genes_lst:
    if gene in gene_symbol_lst:
        print(gene, ' is in genes list')
    else:
        print('*',gene, ' is not in genes list')

APP  is in genes list
SNCA  is in genes list
* IFN  is not in genes list
CCL5  is in genes list
MAPT  is in genes list
* TAU  is not in genes list
BDNF  is in genes list
CCL3  is in genes list
IL4  is in genes list
CCL2  is in genes list
AIF1  is in genes list
IL1B  is in genes list
TARDBP  is in genes list
IL7  is in genes list
IL6  is in genes list
GFAP  is in genes list
* VEGF  is not in genes list
IL10  is in genes list
TNF  is in genes list


In [40]:
# Protein groups
# Amyloid: ihc_a_beta, ab42_over_ab40_ratio, ihc_a_beta_ffpe, ab42_pg_per_mg, ab40_pg_per_mg
# α-synuclein:ihc_a_syn, a_syn_pg_per_mg
# interferon: ifn_g_pg_per_mg
# RANTES: rantes_pg_per_mg
# Tau: ihc_tau2_ffpe, ptau_over_tau_ratio, ptau_ng_per_mg, ihc_at8, ihc_at8_ffpe, tau_ng_per_mg
# Isoprostane: isoprostane_pg_per_mg
# BDNF: bdnf_pg_per_mg
# MIP-1a: mip_1a_pg_per_mg
# Interleukins: il_4_pg_per_mg, il_1b_pg_per_mg, il_7_pg_per_mg, il_6_pg_per_mg, il_10_pg_per_mg
# MCP-1: mcp_1_pg_per_mg
# IBA-1: ihc_iba1_ffpe
# phospho-TDP43: ihc_ptdp_43_ffpe
# GFAP:ihc_gfap_ffpe
# TNF: Tumor Necrosis Factor alpha